In [237]:
import statsmodels.api as sm
import matplotlib 
import scipy.stats
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pingouin
from statsmodels.formula.api import ols

In [47]:
df_data = pd.read_csv('./210312_Fig2C_Data.csv')
df_data.columns

Index(['Unnamed: 0', 'Cluc_W85X_1', 'Cluc_W85X_2', 'Cluc_W85X_3',
       'Cluc_W85X_4', 'CTNNB1_T41A_1', 'CTNNB1_T41A_2', 'CTNNB1_T41A_3',
       'CTNNB1_T41A_4', 'STAT1_Y701C_1', 'STAT1_Y701C_2', 'STAT1_Y701C_3',
       'STAT1_Y701C_4', 'STAT3_Y705C_1', 'STAT3_Y705C_2', 'STAT3_Y705C_3',
       'STAT3_Y705C_4', 'LATS1_T1079A_1', 'LATS1_T1079A_2', 'LATS1_T1079A_3',
       'LATS1_T1079A_4', 'Gluc_R82C_1', 'Gluc_R82C_2', 'Gluc_R82C_3',
       'Gluc_R82C_4', 'CTNNB1_T41I_1', 'CTNNB1_T41I_2', 'CTNNB1_T41I_3',
       'CTNNB1_T41I_4', 'KRAS_D30D_1', 'KRAS_D30D_2', 'KRAS_D30D_3',
       'KRAS_D30D_4', 'KRAS_L56L_1', 'KRAS_L56L_2', 'KRAS_L56L_3',
       'KRAS_L56L_4'],
      dtype='object')

In [240]:
writer = pd.ExcelWriter('Fig2c_statistical_analysis.xlsx', engine='xlsxwriter')

var_list = ['Cluc_W85X','CTNNB1_T41A','STAT1_Y701C','STAT3_Y705C','LATS1_T1079A','Gluc_R82C','CTNNB1_T41I',
           'KRAS_D30D','KRAS_L56L']
for var in var_list:
    zip_order = [(1,1),
                (1,0),
                (0,1),
                (0,0)]
    zip_order = [('+bt1','T'),
                ('+bt1','NT'),
                ('-bt1','T'),
                ('-bt1','NT')]
    df = pd.DataFrame(data=sorted([u + (v,) for i in range(1,5) for u,v in 
                                   zip(zip_order, df_data[var+'_'+str(i)].values)]),
                 columns=['Protein','Targeting','Editing'])
    labels = [str(u)+'_'+str(v) for u,v in zip(df['Protein'], df['Targeting'])]
    df['Labels'] = labels

    # Two-way ANOVA, starting with OLS fit
    model = ols('Editing ~ C(Protein) + C(Targeting) + C(Protein):C(Targeting)', data=df).fit()
    aov_table = sm.stats.anova_lm(model, typ=2)

    # Perform Games-Howell posthoc pairwise analysis
    gh_result = pingouin.pairwise_gameshowell(df,'Editing','Labels')

    aov_table.to_excel(writer, sheet_name=var+'_ANOVA')
    gh_result.to_excel(writer, sheet_name=var+'_GamesHowell')

# Close the Pandas Excel writer
writer.save()

In [241]:
gh_result

,A,B,mean(A),mean(B),diff,se,T,df,pval,hedges
0,+bt1_NT,+bt1_T,0.000486,0.040020,-0.039534,0.003334,-11.859427,3.006132,0.003904,-7.292070
1,+bt1_NT,-bt1_NT,0.000486,0.000266,0.000220,0.000111,1.985041,3.479878,0.341550,1.220553
2,+bt1_NT,-bt1_T,0.000486,0.000418,0.000068,0.000135,0.504632,5.642642,0.900000,0.310286
3,+bt1_T,-bt1_NT,0.040020,0.000266,0.039754,0.003332,11.930961,3.000494,0.003867,7.336055
4,+bt1_T,-bt1_T,0.040020,0.000418,0.039602,0.003333,11.882249,3.003666,0.003897,7.306103
5,-bt1_NT,-bt1_T,0.000266,0.000418,-0.000152,0.000088,-1.730736,3.793449,0.421387,-1.064187


In [218]:
df[(df['Protein'] == 0) & (df['Target'] == 1)]

,Protein,Target,Editing,Labels


In [198]:
np.var(df[(df['Protein'] == 1) & (df['Target'] == 1)]['Editing'].values)

/Users/altaeth/Installations/Anaconda/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3583: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/altaeth/Installations/Anaconda/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:185: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(
/Users/altaeth/Installations/Anaconda/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


nan

In [155]:
np.mean(df[(df['Protein'] == 0)]['Editing'])

0.00072036875

In [156]:
np.mean(df[(df['Protein'] == 1)]['Editing'])

0.13105324375

In [157]:
np.mean(df[(df['Target'] == 0)]['Editing'])

0.0035661625

In [214]:
np.mean(df[(df['Target'] == 1)]['Editing'])

nan

In [235]:
x = []
y = []
x.append(0)
y.append(df[(df['Protein'] == "+") & (df['Targeting'] == "T")]['Editing'].values)

x.append(1)
y.append(df[(df['Protein'] == "+") & (df['Targeting'] == "NT")]['Editing'].values)

x.append(2)
y.append(df[(df['Protein'] == "-") & (df['Targeting'] == "T")]['Editing'].values)

x.append(3)
y.append(df[(df['Protein'] == "-") & (df['Targeting'] == "NT")]['Editing'].values)

w = 0.1

fig, ax = plt.subplots()
ax.bar(x,
       height=[np.mean(yi) for yi in y],
       yerr=[np.std(yi) for yi in y],    # error bars
       capsize=12, # error bar cap width in points
       #width=w,    # bar width
       tick_label=["+/T", "+/NT", "-/T", "-/NT"],
       color=(0,0,0,0),  # face color transparent
       #edgecolor=colors,
       #ecolor=colors,    # error bar colors; setting this raises an error for whatever reason.
       )

for i in range(len(x)):
    # distribute scatter randomly across whole width of bar
    ax.scatter(x[i] + np.random.random(y[i].size) * w - w / 2, y[i])


KeyError: 'Target'

In [201]:


scipy.stats.ttest_ind(df[(df['Protein'] == 1) & (df['Target'] == 1)]['Editing'],
                      df[(df['Protein'] == 1) & (df['Target'] == 0)]['Editing'],
                      equal_var=False)



Ttest_indResult(statistic=nan, pvalue=nan)

In [220]:
labels = [str(u)+'_'+str(v) for u,v in zip(df['Protein'], df['Target'])]
hsd_result = statsmodels.stats.multicomp.pairwise_tukeyhsd(df['Editing'],labels)
hsd_result.summary()

group1,group2,meandiff,p-adj,lower,upper,reject
+_NT,+_T,0.1367,0.001,0.1126,0.1608,True
+_NT,-_NT,-0.0659,0.001,-0.0901,-0.0418,True
+_NT,-_T,-0.0659,0.001,-0.0901,-0.0418,True
+_T,-_NT,-0.2026,0.001,-0.2267,-0.1785,True
+_T,-_T,-0.2026,0.001,-0.2267,-0.1785,True
-_NT,-_T,0.0,0.9,-0.0241,0.0241,False


In [203]:
df

,Protein,Target,Editing
0,+,NT,0.009722
1,+,NT,0.016461
2,+,NT,0.017190
3,+,NT,0.020658
4,+,T,0.363039
5,+,T,0.396324
6,+,T,0.472937
7,+,T,0.545977
8,-,NT,0.000782
9,-,NT,0.000833


In [205]:
list(zip(df['Editing'],labels))

[(0.00972222, '+_NT'),
 (0.01646091, '+_NT'),
 (0.01718984, '+_NT'),
 (0.02065849, '+_NT'),
 (0.36303871, '+_T'),
 (0.39632353, '+_T'),
 (0.472937, '+_T'),
 (0.54597701, '+_T'),
 (0.00078186, '-_NT'),
 (0.00083264, '-_NT'),
 (0.00085543, '-_NT'),
 (0.00107527, '-_NT'),
 (0.0, '-_T'),
 (0.00085179, '-_T'),
 (0.00122026, '-_T'),
 (0.00156495, '-_T')]

In [227]:
result

,A,B,mean(A),mean(B),diff,se,T,df,pval,hedges
0,+_NT,+_T,0.066268,0.202958,-0.136690,0.011485,-11.901295,5.644089,0.001000,-7.317814
1,+_NT,-_NT,0.066268,0.000326,0.065942,0.007028,9.382640,3.000065,0.007798,5.769155
2,+_NT,-_T,0.066268,0.000328,0.065940,0.007028,9.382257,3.000194,0.007797,5.768920
3,+_T,-_NT,0.202958,0.000326,0.202632,0.009084,22.306448,3.000039,0.001000,13.715688
4,+_T,-_T,0.202958,0.000328,0.202630,0.009084,22.306087,3.000116,0.001000,13.715465
5,-_NT,-_T,0.000326,0.000328,-0.000002,0.000046,-0.042749,4.799835,0.900000,-0.026286
